In [1]:
import pandas as pd 
import numpy as np 
import zipfile
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt

/Users/ozge/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Read csv zip
def read_csv_from_zip(file_path, csv_file_within_zip):  
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        with zip_ref.open(csv_file_within_zip) as csv_file:
            df=pd.read_csv(csv_file)
    return df

In [3]:
#Created in Notebook

order_products__train3=pd.read_csv("/Users/ozge/Desktop/Market_Basket_Analysis/notebooks/order_products__train3.txt", sep='\t')

In [7]:
# Dataset
aisles=read_csv_from_zip("/Users/ozge/Desktop/Market_Basket_Analysis/dataset/aisles.csv.zip", "aisles.csv")
departments=read_csv_from_zip("/Users/ozge/Desktop/Market_Basket_Analysis/dataset/departments.csv.zip", "departments.csv")
order_products__prior=read_csv_from_zip("/Users/ozge/Desktop/Market_Basket_Analysis/dataset/order_products__prior.csv.zip", "order_products__prior.csv")
order_products__train=read_csv_from_zip("/Users/ozge/Desktop/Market_Basket_Analysis/dataset/order_products__train.csv.zip", "order_products__train.csv")
orders=read_csv_from_zip("/Users/ozge/Desktop/Market_Basket_Analysis/dataset/orders.csv.zip", "orders.csv")
products=read_csv_from_zip("/Users/ozge/Desktop/Market_Basket_Analysis/dataset/products.csv.zip", "products.csv")
#sample_submission=read_csv_from_zip("/Users/ozge/Desktop/Market_Basket_Analysis/dataset/sample_submission.csv.zip", "sample_submission.csv")




In [47]:
print(aisles.shape)
aisles

(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [46]:
print(departments.shape)
departments

(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [45]:
print(order_products__prior.shape)
order_products__prior

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [6]:
print(order_products__train.shape)
order_products__train

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [49]:
order_products__train.nunique()

order_id             131209
product_id            39123
add_to_cart_order        80
reordered                 2
dtype: int64

In [48]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [52]:
orders.groupby('order_id')['order_id'].count()

order_id
1          1
2          1
3          1
4          1
5          1
          ..
3421079    1
3421080    1
3421081    1
3421082    1
3421083    1
Name: order_id, Length: 3421083, dtype: int64

In [49]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [48]:
order_products__train3

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,112108,train,4,4,10,9.0
2,1,10246,3,0,Organic Celery Hearts,83,4,112108,train,4,4,10,9.0
3,1,49683,4,0,Cucumber Kirby,83,4,112108,train,4,4,10,9.0
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,112108,train,4,4,10,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,Natural Artesian Water,115,7,169679,train,30,0,10,4.0
1384613,3421063,35548,4,1,Twice Baked Potatoes,13,20,169679,train,30,0,10,4.0
1384614,3421070,35951,1,1,Organic Unsweetened Almond Milk,91,16,139822,train,15,6,10,8.0
1384615,3421070,16953,2,1,Creamy Peanut Butter,88,13,139822,train,15,6,10,8.0


#Merge Datasets

order_products__train2=pd.merge(order_products__train, products, on='product_id')
order_products__train3=pd.merge(order_products__train2, orders, on='order_id')

#Save Dataframes

order_products__train3.to_csv("/Users/ozge/Desktop/Market_Basket_Analysis/notebooks/order_products__train3.txt", sep='\t', index=False)


#Predictive Analysis Aisle

aisle_transactions=pd.merge(order_products__train3, aisles, on='aisle_id').groupby('order_id')['aisle'].agg(list)
encoder2=TransactionEncoder()
encoded_aisle=encoder2.fit(aisle_transactions).transform(aisle_transactions)
df_encoded_aisle=pd.DataFrame(encoded_aisle, columns=encoder2.columns_)
df_encoded_aisle.to_csv("/Users/ozge/Desktop/Market_Basket_Analysis/notebooks/df_encoded_aisle.txt", sep='\t',index=False)

df_encoded_aisle

In [10]:
#Apriori
df_encoded_aisle=pd.read_csv("/Users/ozge/Desktop/Market_Basket_Analysis/notebooks/df_encoded_aisle.txt", sep='\t')
frequent_itemset_aisle=pd.DataFrame(apriori(df_encoded_aisle, min_support=0.02, use_colnames=True)).sort_values(by='support', ascending=False)
frequent_itemset_aisle['length']=frequent_itemset_aisle['itemsets'].apply(len)
frequent_itemset_aisle

#frequent_itemset.to_csv("/Users/ozge/Desktop/Market_Basket_Analysis/notebooks/frequent_itemset.txt", sep='\t',index=False)

,support,itemsets,length
27,0.550099,(fresh fruits),1
29,0.450975,(fresh vegetables),1
60,0.383030,(packaged vegetables fruits),1
247,0.327333,"(fresh vegetables, fresh fruits)",2
269,0.287282,"(packaged vegetables fruits, fresh fruits)",2
...,...,...,...
439,0.020067,"(soy lactosefree, soup broth bouillon)",2
140,0.020037,"(milk, canned meals beans)",2
907,0.020029,"(soy lactosefree, frozen produce, fresh fruits...",4
495,0.020029,"(packaged vegetables fruits, fresh fruits, can...",3


In [47]:
#Rule Association Aisle

rules_aisle=association_rules(frequent_itemset_aisle, metric="confidence", min_threshold=0.001)
rules_aisle[(rules_aisle['confidence']<0.7) & (rules_aisle['support']>0.2)].sort_values(by='support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(fresh fruits),(fresh vegetables),0.550099,0.450975,0.327333,0.595043,1.319458,0.079252,1.355760,0.538148
3,(fresh fruits),(packaged vegetables fruits),0.550099,0.383030,0.287282,0.522237,1.363435,0.076577,1.291371,0.592483
4,(packaged vegetables fruits),(fresh vegetables),0.383030,0.450975,0.252361,0.658853,1.460953,0.079624,1.609352,0.511395
5,(fresh vegetables),(packaged vegetables fruits),0.450975,0.383030,0.252361,0.559589,1.460953,0.079624,1.400896,0.574683
8,"(fresh vegetables, fresh fruits)",(packaged vegetables fruits),0.327333,0.383030,0.203919,0.622971,1.626429,0.078541,1.636401,0.572581
9,(packaged vegetables fruits),"(fresh vegetables, fresh fruits)",0.383030,0.327333,0.203919,0.532384,1.626429,0.078541,1.438502,0.624271
10,(fresh vegetables),"(packaged vegetables fruits, fresh fruits)",0.450975,0.287282,0.203919,0.452173,1.573970,0.074362,1.300992,0.664203
11,(fresh fruits),"(packaged vegetables fruits, fresh vegetables)",0.550099,0.252361,0.203919,0.370695,1.468908,0.065095,1.188039,0.709539


In [6]:
# Predictive Analysis of Products

transactions=order_products__train3.groupby('order_id')['product_name'].agg(list)
encoder=TransactionEncoder()
encoded=encoder.fit(transactions).transform(transactions)

df_encoded=pd.DataFrame(encoded, columns=encoder.columns_)
df_encoded



,#2 Coffee Filters,#2 Cone White Coffee Filters,#2 Mechanical Pencils,#4 Natural Brown Coffee Filters,& Go! Hazelnut Spread + Pretzel Sticks,+Energy Black Cherry Vegetable & Fruit Juice,0 Calorie Acai Raspberry Water Beverage,0 Calorie Fuji Apple Pear Water Beverage,0 Calorie Strawberry Dragonfruit Water Beverage,0% Fat Black Cherry Greek Yogurt y,...,with Sweet Cinnamon Bunches Cereal,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
#Drop Column If Frequency Lower then Value

def drop_infrequents(dataframe, frequency_value):
    column_sum=dataframe.sum(axis=0)
    filtered_df=column_sum[column_sum>frequency_value].index
    return dataframe[filtered_df]

In [10]:
#Drop baskets with single items

filtered_df=drop_infrequents(df_encoded, 1)

In [12]:
#Apriori
#df_encoded=pd.read_csv("/Users/ozge/Desktop/Market_Basket_Analysis/notebooks/df_encoded.csv", sep='\t')
frequent_itemset=pd.DataFrame(apriori(filtered_df, min_support=0.02, use_colnames=True))

#frequent_itemset['length']=frequent_itemset['itemsets'].apply(len)
frequent_itemset

#frequent_itemset.to_csv("/Users/ozge/Desktop/Market_Basket_Analysis/notebooks/frequent_itemset.txt", sep='\t',index=False)


In [31]:
frequent_itemset[frequent_itemset['length']>1].sort_values(by='support', ascending=False)

,support,itemsets,length
29,0.023428,"(Organic Strawberries, Bag of Organic Bananas)",2


In [ ]:
#Association Rules

rules=association_rules(frequent_itemset, metric="confidence", min_threshold=0.001)

In [ ]:
#Data Analysis: Most Frequent Items

